In [1]:
import transformers
import torch
import pandas as pd
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, Trainer, TrainingArguments, BartForConditionalGeneration
from transformers.modeling_outputs import BaseModelOutput
from torch.optim import Adam
from accelerate import Accelerator
import wandb
from tqdm import tqdm
from torch.optim import AdamW

In [2]:
from evaluate import load
from rouge_score import rouge_scorer
from bert_score import score as bert_score

# meteor = load("meteor")
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [3]:
accelerator = Accelerator()

In [4]:
!wandb login --relogin --verify f59d448beb3315f3efbc5a0a80d9d2c346926308

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/dhruv/.netrc
wandb: Currently logged in as: noel22338 (nlp_project_team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
training_data = pd.read_csv('../Dataset/train.csv')
testing_data = pd.read_csv('../Dataset/test.csv')
validation_data = pd.read_csv('../Dataset/validation.csv')

In [6]:
print(training_data.head())

                                          hatespeech       csType  \
0  Maybe the UN could talk to those asian and afr...  Informative   
1  Maybe the UN could talk to those asian and afr...  Questioning   
2  Maybe the UN could talk to those asian and afr...   Denouncing   
3  Maybe the UN could talk to those asian and afr...     Positive   
4  Juice Jews are worse that nukes. Caption to a ...  Informative   

                                       counterspeech Suggest  Relevance  \
0  The us is the second most polluting country in...       3        4.0   
1  Doesn't everyone on the planet have a responsi...                3.0   
2  The world would be a better place if people we...       1        1.0   
3  You're right, ocean pollution is one of the ma...       3        4.0   
4  Anti-semitism is a serious problem that we nee...       2        3.0   

   Aggressive  Complexity  Comments source  \
0         2.0         3.0       NaN  Human   
1         2.0         2.0       NaN  Human

In [7]:
columns = training_data.columns
print(columns)

Index(['hatespeech', 'csType', 'counterspeech', 'Suggest', 'Relevance',
       'Aggressive', 'Complexity', 'Comments', 'source', 'claim',
       'centralTopic', 'speakerIntent', 'targetGroup', 'relevantPowerDynamics',
       'hatespeechImplication', 'targetGroupEmotionalReaction',
       'targetGroupCognitiveReaction', 'hatespeechOffensiveness', 'id',
       'is_high_quality', 'hs_id', 'hatespeechTarget', 'powerDynamics',
       'prompt_offensiveness', 'prompt_target_group', 'prompt_speaker_intent',
       'prompt_power_dynamics', 'prompt_implication',
       'prompt_emotional_reaction', 'prompt_cognitive_reaction',
       'prompt_cs_generation'],
      dtype='object')


In [8]:
class DialoGPTDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")
        self.bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        # Tokenize hate speech
        hate_inputs = self.tokenizer(
            row["hatespeech"],
            return_tensors='pt',
            max_length=128,
            truncation=True,
            padding="max_length"
        )
        # Tokenize counterspeech
        counter_inputs = self.bart_tokenizer(
            row["counterspeech"],
            return_tensors='pt',
            max_length=128,
            truncation=True,
            padding="max_length"
        )

        # Define intent categories
        categories = {
            'informative': 0,
            'questioning': 1,
            'denouncing': 2,
            'positive': 3,
            'humor': 4
        }

        return {
            'input_ids': hate_inputs['input_ids'].squeeze(0),
            'attention_mask': hate_inputs['attention_mask'].squeeze(0),
            'counter_speech': counter_inputs['input_ids'].squeeze(0),
            'intent_id': torch.tensor(categories[row["csType"].lower()], dtype=torch.long)
        }

    def __len__(self):
        return len(self.data)

In [9]:
train_dataset = DialoGPTDataset(training_data)
test_dataset = DialoGPTDataset(testing_data)
validation_dataset = DialoGPTDataset(validation_data)

print(len(train_dataset))
print(len(test_dataset))
print(len(validation_dataset))

print(train_dataset[0])


9532
2971
1470
{'input_ids': tensor([  101,  2672,  1996,  4895,  2071,  2831,  2000,  2216,  4004,  1998,
         3060,  3741,  3625,  2005,  3938,  1009,  1997,  1996, 10796,  1999,
         1996, 17401,  2612,  1997, 22604,  2006,  2023, 14636,  2055,  4785,
         2689,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,

In [10]:
class FeatureEncoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, next_input):
        super(FeatureEncoder, self).__init__()
        self.model = AutoModel.from_pretrained('GroNLP/hateBERT')
        self.hidden_dim = hidden_dim
        self.output_size = next_input

        self.informative_head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.output_size),
            torch.nn.ReLU(),
        )

        self.questioning_head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.output_size),
            torch.nn.ReLU(),
        )

        self.denouncing_head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.output_size),
            torch.nn.ReLU(),
        )

        self.positive_head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.output_size),
            torch.nn.ReLU(),
        )

        self.humor_head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.output_size),
            torch.nn.ReLU(),
        )


    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        hate_speech_h = outputs.last_hidden_state[:, 0, :]

        informative_e = self.informative_head(hate_speech_h)
        questioning_e = self.questioning_head(hate_speech_h)
        denouncing_e = self.denouncing_head(hate_speech_h)
        positive_e = self.positive_head(hate_speech_h)
        humor_e = self.humor_head(hate_speech_h)

        return informative_e, questioning_e, denouncing_e, positive_e, humor_e, hate_speech_h

In [11]:
feature_encoder = FeatureEncoder(input_dim=128, hidden_dim=768, next_input=100)
print(feature_encoder(train_dataset[0]['input_ids'].unsqueeze(0), train_dataset[0]['attention_mask'].unsqueeze(0)))

(tensor([[0.0000, 0.0000, 0.0000, 0.1041, 0.1391, 0.2064, 0.0000, 0.0000, 0.1552,
         0.0000, 0.0000, 0.0000, 0.0000, 0.4886, 0.1636, 0.0000, 0.0000, 0.0977,
         0.0000, 0.2259, 0.0000, 0.0000, 0.3120, 0.2144, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0521, 0.0000, 0.1998, 0.0000, 0.0000, 0.0322, 0.1853, 0.0669,
         0.0000, 0.0000, 0.0000, 0.3507, 0.0000, 0.0000, 0.0000, 0.2598, 0.1648,
         0.0390, 0.0288, 0.0000, 0.0000, 0.0000, 0.0000, 0.3204, 0.0173, 0.0625,
         0.0778, 0.7104, 0.0812, 0.1457, 0.3830, 0.0000, 0.1012, 0.1235, 0.0260,
         0.0000, 0.2348, 0.0000, 0.1559, 0.0000, 0.1859, 0.3386, 0.0000, 0.0000,
         0.5267, 0.0749, 0.2583, 0.3908, 0.0000, 0.0000, 0.0000, 0.4191, 0.0000,
         0.0000, 0.4671, 0.0000, 0.0000, 0.0653, 0.3277, 0.0000, 0.1911, 0.0000,
         0.3217, 0.3876, 0.0279, 0.0799, 0.0922, 0.0929, 0.0000, 0.1722, 0.0000,
         0.3725]], grad_fn=<ReluBackward0>), tensor([[0.1835, 0.1692, 0.0791, 0.0622, 0.3001, 0.0000, 0.0000

In [12]:
class CounterSpeechNetwork(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, encoder_output, max_length):
        super(CounterSpeechNetwork, self).__init__()

        self.feature_encoder = FeatureEncoder(input_dim, hidden_dim, encoder_output)

        self.informative_decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
        self.questioning_decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
        self.denouncing_decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
        self.positive_decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
        self.humor_decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

        self.informative_fusion = torch.nn.Linear(hidden_dim + encoder_output, self.informative_decoder.config.d_model)
        self.questioning_fusion = torch.nn.Linear(hidden_dim + encoder_output, self.questioning_decoder.config.d_model)
        self.denouncing_fusion = torch.nn.Linear(hidden_dim + encoder_output, self.denouncing_decoder.config.d_model)
        self.positive_fusion = torch.nn.Linear(hidden_dim + encoder_output, self.positive_decoder.config.d_model)
        self.humor_fusion = torch.nn.Linear(hidden_dim + encoder_output, self.humor_decoder.config.d_model)

        self.tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
        self.max_length = max_length

    def forward(self, input_ids, attention_mask, intent_id, counter_speech=None):
        informative_e, questioning_e, denouncing_e, positive_e, humor_e, hate_speech_h = self.feature_encoder(input_ids, attention_mask)

        batch_size = input_ids.size(0)

        fused = torch.zeros(batch_size, 1, self.informative_decoder.config.d_model, device=input_ids.device)

        for i in range(batch_size):
            if intent_id[i] == 0:
                fused[i] = self.informative_fusion(torch.cat((hate_speech_h[i], informative_e[i]), dim=-1)).unsqueeze(0)
            elif intent_id[i] == 1:
                fused[i] = self.questioning_fusion(torch.cat((hate_speech_h[i], questioning_e[i]), dim=-1)).unsqueeze(0)
            elif intent_id[i] == 2:
                fused[i] = self.denouncing_fusion(torch.cat((hate_speech_h[i], denouncing_e[i]), dim=-1)).unsqueeze(0)
            elif intent_id[i] == 3:
                fused[i] = self.positive_fusion(torch.cat((hate_speech_h[i], positive_e[i]), dim=-1)).unsqueeze(0)
            elif intent_id[i] == 4:
                fused[i] = self.humor_fusion(torch.cat((hate_speech_h[i], humor_e[i]), dim=-1)).unsqueeze(0)
            else:
                raise ValueError(f"Invalid intent_id: {intent_id[i]}")

        if counter_speech is not None:
            losses = []
            for i in range(batch_size):
                if intent_id[i] == 0:
                    output = self.informative_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0))
                elif intent_id[i] == 1:
                    output = self.questioning_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0))
                elif intent_id[i] == 2:
                    output = self.denouncing_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0))
                elif intent_id[i] == 3:
                    output = self.positive_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0))
                elif intent_id[i] == 4:
                    output = self.humor_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0))
                losses.append(output.loss)
            avg_loss = sum(losses) / len(losses)  # Average loss across the batch
            return None, avg_loss  # No decoded text during training
        else:
            decoded_texts = []
            for i in range(batch_size):
                if intent_id[i] == 0:
                    output = self.informative_decoder.generate(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), max_length=self.max_length, num_beams=4, early_stopping=True)
                elif intent_id[i] == 1:
                    output = self.questioning_decoder.generate(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), max_length=self.max_length, num_beams=4, early_stopping=True)
                elif intent_id[i] == 2:
                    output = self.denouncing_decoder.generate(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), max_length=self.max_length, num_beams=4, early_stopping=True)
                elif intent_id[i] == 3:
                    output = self.positive_decoder.generate(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), max_length=self.max_length, num_beams=4, early_stopping=True)
                elif intent_id[i] == 4:
                    output = self.humor_decoder.generate(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), max_length=self.max_length, num_beams=4, early_stopping=True)
                decoded_texts.append(self.tokenizer.decode(output[0], skip_special_tokens=True))
            return decoded_texts, None  # Decoded text during inference, no loss

In [13]:
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)

counterspeechnetwork = CounterSpeechNetwork(input_dim=128, hidden_dim=768, encoder_output=256, max_length=50)

for batch in train_dataloader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    intent_id = batch['intent_id']
    counter_speech = batch['counter_speech']

    print(counterspeechnetwork(input_ids, attention_mask, intent_id, counter_speech))
    break

(None, tensor(15.3572, grad_fn=<DivBackward0>))


In [14]:
model = CounterSpeechNetwork(input_dim=128, hidden_dim=768, encoder_output=256, max_length=50)
optimizer = AdamW(model.parameters(), lr=5e-5)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=32, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 10

for epoch in range(epochs):
    model.train()
    total_train_loss = 0.0

    # Wrap train_dataloader with tqdm for training progress
    train_loop = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs} [Train]", leave=False)
    for batch in train_loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        counter_speech = batch['counter_speech'].to(device)
        intent_ids = batch['intent_id'].to(device)

        optimizer.zero_grad()

        # Process the entire batch at once
        _, loss = model(input_ids, attention_mask, intent_ids, counter_speech)

        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

        # Update tqdm with current batch loss
        train_loop.set_postfix({'batch_loss': loss.item(), 'avg_loss': total_train_loss / (train_loop.n + 1)})

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    total_val_loss = 0.0

    # Wrap validation_dataloader with tqdm for validation progress
    val_loop = tqdm(validation_dataloader, desc=f"Epoch {epoch + 1}/{epochs} [Validation]", leave=False)
    with torch.no_grad():
        for batch in val_loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            counter_speech = batch['counter_speech'].to(device)
            intent_ids = batch['intent_id'].to(device)

            # Process the entire batch at once
            _, loss = model(input_ids, attention_mask, intent_ids, counter_speech)

            total_val_loss += loss.item()

            # Update tqdm with current batch loss
            val_loop.set_postfix({'batch_loss': loss.item(), 'avg_loss': total_val_loss / (val_loop.n + 1)})

    avg_val_loss = total_val_loss / len(validation_dataloader)
    print(f"Epoch {epoch + 1}/{epochs} | Validation Loss: {avg_val_loss:.4f}")

Epoch 1/10 | Train Loss: 2.1973


Epoch 1/10 | Validation Loss: 0.7590


Epoch 2/10 | Train Loss: 0.8208


Epoch 2/10 | Validation Loss: 0.7166


Epoch 3/10 | Train Loss: 0.7238


...

Epoch 8/10 | Validation Loss: 0.6950


Epoch 9/10 | Train Loss: 0.4509


Epoch 9/10 | Validation Loss: 0.7109


Epoch 10/10 | Train Loss: 0.4156


Epoch 10/10 | Validation Loss: 0.7229


In [15]:
torch.save(model.state_dict(), 'architecture_parameters.pth')

In [16]:
# Test evaluation
model.eval()
test_predictions = []
test_references = []

# Wrap test_dataloader with tqdm for test progress
test_loop = tqdm(test_dataloader, desc="Test Evaluation", leave=True)
with torch.no_grad():
    for batch in test_loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        counter_speech = batch['counter_speech'].to(device)  # Reference texts
        intent_ids = batch['intent_id'].to(device)

        # Generate predictions without providing counter_speech
        predictions, _ = model(input_ids, attention_mask, intent_ids)

        # Decode reference texts
        references = [model.tokenizer.decode(cs, skip_special_tokens=True) for cs in counter_speech]

        test_predictions.extend(predictions)
        test_references.extend(references)

        # Update tqdm with progress stats (e.g., number of predictions)
        test_loop.set_postfix({'predictions': len(test_predictions)})

# Final output
print(f"Generated {len(test_predictions)} test predictions")

Test Evaluation: 100%|██████████████████████████████████████████████████████████████████| 93/93 [09:44<00:00,  6.28s/it, predictions=2971]

Generated 2971 test predictions


In [ ]:
P, R, F1 = bert_score(test_predictions, test_references, lang="en", verbose=True)

# Final output
print(f"Generated {len(test_predictions)} test predictions")
print(f"BERTScore - Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
for pred, ref in zip(test_predictions, test_references):
    scores = scorer.score(ref, pred)  # ROUGE scores for each prediction-reference pair
    rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
    rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
    rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

# Compute average ROUGE scores
avg_rouge1 = sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1'])
avg_rouge2 = sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2'])
avg_rougeL = sum(rouge_scores['rougeL']) / len(rouge_scores['rougeL'])

print(f"ROUGE - Rouge1: {avg_rouge1:.4f}, Rouge2: {avg_rouge2:.4f}, RougeL: {avg_rougeL:.4f}")

In [ ]:
for pred, ref in zip(test_predictions, test_references):
    print(f'Generated: {pred}')
    print(f'Actual: {ref}')
    print()